**Code Written by:** <br>
Ricardo G. Mora, Jr.  10/16/2022 <br><br>
**Description:** <br>
This notebook demonstrates several ways to use Pandas data frames to analyze the academic performance for the schools of fictitious PyCity.  The input data comes from the csv files "students_complete.csv" and "schools_complete.csv".  There is not an output data file.  All output is contained within this notebook and appears below each corresponding code cell.

## Initial Setup and Reading of Files
* Note: Use data files "students_complete.csv" and "schools_complete.csv" in folder "Resources"

In [1]:
# Dependencies:
import pandas as pd
import numpy as np

# Set the input files:
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames:
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Check for missing data:
student_data.count()
# All columns have the same amount of data: Data cleaning not needed.

Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
dtype: int64

* Combine the data into a single dataset

In [2]:
# Left join school data to student data:  
combined_data = pd.merge(student_data, school_data, how="left", on="school_name")

# Examine data:
combined_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Create an overview table that summarizes key metrics for the entire district, including:
    * Total Schools
    * Total Students
    * Total Budget
    * Average Math Score
    * Average Reading Score
    * % of Students Passing Math (70 or greater)
    * % of Students Passing Reading (70 or greater)
    * % of Students Passing Overall (Passing Math **and** Reading)

In [3]:
# Get/Calculate the summary statistics:
school_count = combined_data["School ID"].nunique()
student_count = combined_data["Student ID"].nunique()
total_school_budgets = school_data["budget"].sum()
avg_math_score = student_data["math_score"].mean()
avg_reading_score = student_data["reading_score"].mean()
percent_passing_math = student_data.loc[student_data["math_score"] >= 70, "math_score"].count() / student_count
percent_passing_reading = student_data.loc[student_data["reading_score"] >= 70, "reading_score"].count() / student_count
percent_passing_both = student_data.loc[(student_data["math_score"] >= 70) & (student_data["reading_score"] >= 70), "Student ID"].count() / student_count

# Put into dataframe:
District_Summary_DF = pd.DataFrame({
    "District": "PyCity",
    "Total Schools": [school_count],   # (Must convert at least one variable to a list since all are scalars.)
    "Total Students": student_count,
    "Total Budget": total_school_budgets,
    "Average Math Score": avg_math_score,
    "Average Reading Score": avg_reading_score,
    "% Passing Math": percent_passing_math,
    "% Passing Reading": percent_passing_reading,
    "% Overall Passing": percent_passing_both
})

# Format dataframe values:
District_Summary_DF["Total Students"] = District_Summary_DF["Total Students"].map("{:,}".format)
District_Summary_DF["Total Budget"] = District_Summary_DF["Total Budget"].map("${:,.2f}".format)
District_Summary_DF["Average Math Score"] = District_Summary_DF["Average Math Score"].map("{:,.2f}".format)
District_Summary_DF["Average Reading Score"] = District_Summary_DF["Average Reading Score"].map("{:,.2f}".format)
District_Summary_DF["% Passing Math"] = District_Summary_DF["% Passing Math"].map("{:.2%}".format)
District_Summary_DF["% Passing Reading"] = District_Summary_DF["% Passing Reading"].map("{:.2%}".format)
District_Summary_DF["% Overall Passing"] = District_Summary_DF["% Overall Passing"].map("{:.2%}".format)

# Display dataframe:
styles = [dict(selector="caption", props=[("font-size","150%"), ("font-weight","bold"), ("text-align", "center"),
                                          ("background-color", "whitesmoke")])]
title = "District Summary"
District_Summary_DF.set_index("District").style.set_caption(title).set_table_styles(styles)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
District,,,,,,,,
PyCity,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Passing Overall (Passing Math **and** Reading.)

In [4]:
# Calculate the summary statistics:
per_student_budgets = school_data["budget"] / school_data["size"].astype("float")
avg_math_scores = combined_data.groupby("School ID").mean()["math_score"]
avg_reading_scores = combined_data.groupby("School ID").mean()["reading_score"]
percent_math_passers = combined_data[combined_data["math_score"] >= 70].groupby("School ID").count()["math_score"] / school_data["size"]
percent_reading_passers = combined_data[combined_data["reading_score"] >= 70].groupby("School ID").count()["reading_score"] / school_data["size"]
percent_overall_passers = combined_data[(combined_data["math_score"] >= 70) & (combined_data["reading_score"] >= 70)].groupby("School ID").count()["math_score"] / school_data["size"]

# Put into dataframe:
Expanded_School_Data_DF = pd.DataFrame({
    "School Name": school_data["school_name"],
    "School Type": school_data["type"],
    "Total Students": school_data["size"],
    "Total School Budget": school_data["budget"],
    "Per Student Budget": per_student_budgets,
    "Average Math Score": avg_math_scores,
    "Average Reading Score": avg_reading_scores,
    "% Passing Math": percent_math_passers,
    "% Passing Reading": percent_reading_passers,
    "% Overall Passing": percent_overall_passers
})

# Create a copy of the dataframe with formatted values:
Formatted_School_Data_DF = Expanded_School_Data_DF.loc[:, ["School Name", "School Type", "Total Students"]]
Formatted_School_Data_DF["Total School Budget"] = Expanded_School_Data_DF.loc[:, "Total School Budget"].map("${:,.2f}".format)
Formatted_School_Data_DF["Per Student Budget"] = Expanded_School_Data_DF.loc[:, "Per Student Budget"].map("${:,.2f}".format)
Formatted_School_Data_DF["Average Math Score"] = Expanded_School_Data_DF.loc[:, "Average Math Score"].map("{:.2f}".format)
Formatted_School_Data_DF["Average Reading Score"] = Expanded_School_Data_DF.loc[:, "Average Reading Score"].map("{:.2f}".format)
Formatted_School_Data_DF["% Passing Math"] = Expanded_School_Data_DF.loc[:, "% Passing Math"].map("{:.2%}".format)
Formatted_School_Data_DF["% Passing Reading"] = Expanded_School_Data_DF.loc[:, "% Passing Reading"].map("{:.2%}".format)
Formatted_School_Data_DF["% Overall Passing"] = Expanded_School_Data_DF.loc[:, "% Overall Passing"].map("{:.2%}".format)

# Display formatted dataframe:
styles = [dict(selector="caption", props=[("font-size","150%"), ("font-weight","bold"), ("text-align", "center"),
                                          ("background-color", "whitesmoke"), ("border-style","solid"), ("border-width", "1px")])]
title = "Summary For Each School"
Formatted_School_Data_DF.set_index("School Name").sort_index().style.set_caption(title).set_table_styles(styles)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Top 5 Performing Schools (By % Overall Passing)

In [5]:
# Display the sorted dataframe:
styles = [dict(selector="caption", props=[("font-size","150%"), ("font-weight","bold"), ("text-align", "center"),
                                          ("background-color", "whitesmoke"), ("border-style","solid"), ("border-width", "1px")])]
title = "Top 5 Performing Schools (by % Overall Passing)"
Formatted_School_Data_DF.set_index("School Name").sort_values("% Overall Passing", ascending=False).head()\
                        .style.set_caption(title).set_table_styles(styles)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Bottom 5 Performing Schools (By % Overall Passing)

In [6]:
# Display the sorted dataframe:
styles = [dict(selector="caption", props=[("font-size","150%"), ("font-weight","bold"), ("text-align", "center"),
                                          ("background-color", "whitesmoke"), ("border-style","solid"), ("border-width", "1px")])]
title = "Bottom 5 Performing Schools (by % Overall Passing)"
Formatted_School_Data_DF.set_index("School Name").sort_values("% Overall Passing", ascending=True).head()\
                        .style.set_caption(title).set_table_styles(styles)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Average Math Scores by Grade Level at Each School

In [7]:
# Calculate the average math scores for each grade:
avg_math_scores_9th = combined_data[combined_data["grade"] == "9th"].groupby("School ID").mean()["math_score"]
avg_math_scores_10th = combined_data[combined_data["grade"] == "10th"].groupby("School ID").mean()["math_score"]
avg_math_scores_11th = combined_data[combined_data["grade"] == "11th"].groupby("School ID").mean()["math_score"]
avg_math_scores_12th = combined_data[combined_data["grade"] == "12th"].groupby("School ID").mean()["math_score"]


# Put into dataframe:
Math_Scores_By_Grade_DF = pd.DataFrame({
    "School Name": school_data["school_name"],
    "9th": avg_math_scores_9th,
    "10th": avg_math_scores_10th,
    "11th": avg_math_scores_11th,
    "12th": avg_math_scores_12th
})

# Make a column that shows which grade performs best in each school:
def best(row):
    highest = max([x for x in row if type(x) == int or type(x) == float])
    value = ""
    if row["9th"] == highest:
        value = value + " 9th"
    if row["10th"] == highest:
        value = value + " 10th"
    if row["11th"] == highest:
        value = value + " 11th"
    if row["12th"] == highest:
        value = value + " 12th"
    return value     # (doing it this way will show multiple grades if there is a tie)

Math_Scores_By_Grade_DF["Best Grade"] = Math_Scores_By_Grade_DF.apply(best, axis=1)

# Make a column that shows which grade performs worst in each school:
def worst(row):
    lowest = min([x for x in row if type(x) == int or type(x) == float])
    value = ""
    if row["9th"] == lowest:
        value = value + " 9th"
    if row["10th"] == lowest:
        value = value + " 10th"
    if row["11th"] == lowest:
        value = value + " 11th"
    if row["12th"] == lowest:
        value = value + " 12th"
    return value     # (doing it this way will show multiple grades if there is a tie)

Math_Scores_By_Grade_DF["Worst Grade"] = Math_Scores_By_Grade_DF.apply(worst, axis=1)

# Make a column for the greatest change accross the grades for each school:
Math_Scores_By_Grade_DF["By How Much"] = Math_Scores_By_Grade_DF.max(axis=1) - Math_Scores_By_Grade_DF.min(axis=1)

# Format dataframe values:
Math_Scores_By_Grade_DF["9th"] = Math_Scores_By_Grade_DF["9th"].map("{:.2f}".format)
Math_Scores_By_Grade_DF["10th"] = Math_Scores_By_Grade_DF["10th"].map("{:.2f}".format)
Math_Scores_By_Grade_DF["11th"] = Math_Scores_By_Grade_DF["11th"].map("{:.2f}".format)
Math_Scores_By_Grade_DF["12th"] = Math_Scores_By_Grade_DF["12th"].map("{:.2f}".format)
Math_Scores_By_Grade_DF["By How Much"] = Math_Scores_By_Grade_DF["By How Much"].map("{:.2f}".format)

# Display dataframe:
styles = [dict(selector="caption", props=[("font-size","150%"), ("font-weight","bold"), ("text-align", "center"),
                                          ("background-color", "whitesmoke"), ("border-style","solid"), ("border-width", "1px")])]
title = "Each School's Average Math Score by Grade Level"
Math_Scores_By_Grade_DF.set_index("School Name").sort_index().style.set_caption(title).set_table_styles(styles)

,9th,10th,11th,12th,Best Grade,Worst Grade,By How Much
School Name,,,,,,,
Bailey High School,77.08,77.00,77.52,76.49,11th,12th,1.02
Cabrera High School,83.09,83.15,82.77,83.28,12th,11th,0.51
Figueroa High School,76.40,76.54,76.88,77.15,12th,9th,0.75
Ford High School,77.36,77.67,76.92,76.18,10th,12th,1.49
Griffin High School,82.04,84.23,83.84,83.36,10th,9th,2.19
Hernandez High School,77.44,77.34,77.14,77.19,9th,11th,0.30
Holden High School,83.79,83.43,85.00,82.86,11th,12th,2.14
Huang High School,77.03,75.91,76.45,77.23,12th,10th,1.32
Johnson High School,77.19,76.69,77.49,76.86,11th,10th,0.80


## Average Reading Scores by Grade Level at Each School

In [8]:
# Calculate the average reading scores for each grade:
avg_reading_scores_9th = combined_data[combined_data["grade"] == "9th"].groupby("School ID").mean()["reading_score"]
avg_reading_scores_10th = combined_data[combined_data["grade"] == "10th"].groupby("School ID").mean()["reading_score"]
avg_reading_scores_11th = combined_data[combined_data["grade"] == "11th"].groupby("School ID").mean()["reading_score"]
avg_reading_scores_12th = combined_data[combined_data["grade"] == "12th"].groupby("School ID").mean()["reading_score"]


# Put into dataframe:
Reading_Scores_By_Grade_DF = pd.DataFrame({
    "School Name": school_data["school_name"],
    "9th": avg_reading_scores_9th,
    "10th": avg_reading_scores_10th,
    "11th": avg_reading_scores_11th,
    "12th": avg_reading_scores_12th
})

# Make a column that shows which grade performs best in each school:
def best(row):
    highest = max([x for x in row if type(x) == int or type(x) == float])
    value = ""
    if row["9th"] == highest:
        value = value + " 9th"
    if row["10th"] == highest:
        value = value + " 10th"
    if row["11th"] == highest:
        value = value + " 11th"
    if row["12th"] == highest:
        value = value + " 12th"
    return value     # (doing it this way will show multiple grades if there is a tie)

Reading_Scores_By_Grade_DF["Best Grade"] = Reading_Scores_By_Grade_DF.apply(best, axis=1)

# Make a column that shows which grade performs worst in each school:
def worst(row):
    lowest = min([x for x in row if type(x) == int or type(x) == float])
    value = ""
    if row["9th"] == lowest:
        value = value + " 9th"
    if row["10th"] == lowest:
        value = value + " 10th"
    if row["11th"] == lowest:
        value = value + " 11th"
    if row["12th"] == lowest:
        value = value + " 12th"
    return value     # (doing it this way will show multiple grades if there is a tie)

Reading_Scores_By_Grade_DF["Worst Grade"] = Reading_Scores_By_Grade_DF.apply(worst, axis=1)


# Find the greatest change accross the grades for each school:
Reading_Scores_By_Grade_DF["By How Much"] = Reading_Scores_By_Grade_DF.max(axis=1) - Reading_Scores_By_Grade_DF.min(axis=1)

# Format dataframe values:
Reading_Scores_By_Grade_DF["9th"] = Reading_Scores_By_Grade_DF["9th"].map("{:.2f}".format)
Reading_Scores_By_Grade_DF["10th"] = Reading_Scores_By_Grade_DF["10th"].map("{:.2f}".format)
Reading_Scores_By_Grade_DF["11th"] = Reading_Scores_By_Grade_DF["11th"].map("{:.2f}".format)
Reading_Scores_By_Grade_DF["12th"] = Reading_Scores_By_Grade_DF["12th"].map("{:.2f}".format)
Reading_Scores_By_Grade_DF["By How Much"] = Reading_Scores_By_Grade_DF["By How Much"].map("{:.2f}".format)

# Display dataframe:
styles = [dict(selector="caption", props=[("font-size","150%"), ("font-weight","bold"), ("text-align", "center"),
                                          ("background-color", "whitesmoke"), ("border-style","solid"), ("border-width", "1px")])]
title = "Each School's Average Reading Score by Grade Level"
Reading_Scores_By_Grade_DF.set_index("School Name").sort_index().style.set_caption(title).set_table_styles(styles)

,9th,10th,11th,12th,Best Grade,Worst Grade,By How Much
School Name,,,,,,,
Bailey High School,81.30,80.91,80.95,80.91,9th,10th,0.40
Cabrera High School,83.68,84.25,83.79,84.29,12th,9th,0.61
Figueroa High School,81.20,81.41,80.64,81.38,10th,11th,0.77
Ford High School,80.63,81.26,80.40,80.66,10th,11th,0.86
Griffin High School,83.37,83.71,84.29,84.01,11th,9th,0.92
Hernandez High School,80.87,80.66,81.40,80.86,11th,10th,0.74
Holden High School,83.68,83.32,83.82,84.70,12th,10th,1.37
Huang High School,81.29,81.51,81.42,80.31,10th,12th,1.21
Johnson High School,81.26,80.77,80.62,81.23,9th,11th,0.64


## Average Math Scores by Gender at Each School

In [9]:
# Calculate the average math scores for each gender:
avg_math_scores_f = combined_data[combined_data["gender"] == "F"].groupby("School ID").mean()["math_score"]
avg_math_scores_m = combined_data[combined_data["gender"] == "M"].groupby("School ID").mean()["math_score"]

# Put into dataframe:
Math_Scores_By_Gender_DF = pd.DataFrame({
    "School Name": school_data["school_name"],
    "F": avg_math_scores_f,
    "M": avg_math_scores_m
})

# Find who has the best score (boys or girls) and by how much:
Math_Scores_By_Gender_DF["Who's Better"] = np.where(Math_Scores_By_Gender_DF["F"] > Math_Scores_By_Gender_DF["M"], "F", "M")
Math_Scores_By_Gender_DF["How Much Better"] = Math_Scores_By_Gender_DF.max(axis=1) - Math_Scores_By_Gender_DF.min(axis=1)

# Format dataframe values:
Math_Scores_By_Gender_DF["F"] = Math_Scores_By_Gender_DF["F"].map("{:.2f}".format)
Math_Scores_By_Gender_DF["M"] = Math_Scores_By_Gender_DF["M"].map("{:.2f}".format)
Math_Scores_By_Gender_DF["How Much Better"] = Math_Scores_By_Gender_DF["How Much Better"].map("{:.2f}".format)

# Display dataframe:
styles = [dict(selector="caption", props=[("font-size","150%"), ("font-weight","bold"), ("text-align", "center"),
                                          ("background-color", "whitesmoke"), ("border-style","solid"), ("border-width", "1px")])]
title = "Each School's Average Math Score by Gender"
Math_Scores_By_Gender_DF.set_index("School Name").sort_index().style.set_caption(title).set_table_styles(styles)

,F,M,Who's Better,How Much Better
School Name,,,,
Bailey High School,77.00,77.10,M,0.10
Cabrera High School,83.07,83.06,F,0.01
Figueroa High School,76.82,76.61,F,0.22
Ford High School,77.16,77.05,F,0.11
Griffin High School,83.29,83.41,M,0.11
Hernandez High School,77.52,77.05,F,0.48
Holden High School,83.60,84.01,M,0.41
Huang High School,76.51,76.74,M,0.22
Johnson High School,77.17,76.98,F,0.19


## Average Reading Scores by Gender at Each School

In [10]:
# Calculate the average reading scores for each gender:
avg_reading_scores_f = combined_data[combined_data["gender"] == "F"].groupby("School ID").mean()["reading_score"]
avg_reading_scores_m = combined_data[combined_data["gender"] == "M"].groupby("School ID").mean()["reading_score"]

# Put into dataframe:
Reading_Scores_By_Gender_DF = pd.DataFrame({
    "School Name": school_data["school_name"],
    "F": avg_reading_scores_f,
    "M": avg_reading_scores_m
})

# Find who has the best score (boys or girls) and by how much:
Reading_Scores_By_Gender_DF["Who's Better"] = np.where(Reading_Scores_By_Gender_DF["F"] > Reading_Scores_By_Gender_DF["M"], "F", "M")
Reading_Scores_By_Gender_DF["How Much Better"] = Reading_Scores_By_Gender_DF.max(axis=1) - Reading_Scores_By_Gender_DF.min(axis=1)

# Format dataframe values:
Reading_Scores_By_Gender_DF["F"] = Reading_Scores_By_Gender_DF["F"].map("{:.2f}".format)
Reading_Scores_By_Gender_DF["M"] = Reading_Scores_By_Gender_DF["M"].map("{:.2f}".format)
Reading_Scores_By_Gender_DF["How Much Better"] = Reading_Scores_By_Gender_DF["How Much Better"].map("{:.2f}".format)

# Display dataframe:
styles = [dict(selector="caption", props=[("font-size","150%"), ("font-weight","bold"), ("text-align", "center"),
                                          ("background-color", "whitesmoke"), ("border-style","solid"), ("border-width", "1px")])]
title = "Each School's Average Reading Score by Gender"
Reading_Scores_By_Gender_DF.set_index("School Name").sort_index().style.set_caption(title).set_table_styles(styles)

,F,M,Who's Better,How Much Better
School Name,,,,
Bailey High School,81.46,80.59,F,0.87
Cabrera High School,83.99,83.96,F,0.03
Figueroa High School,81.18,81.14,F,0.05
Ford High School,80.73,80.77,M,0.04
Griffin High School,83.82,83.82,M,0.00
Hernandez High School,81.06,80.80,F,0.25
Holden High School,83.69,83.95,M,0.27
Huang High School,81.45,80.93,F,0.52
Johnson High School,81.33,80.60,F,0.72


## Average Scores and Passing Rates by School Spending

In [11]:
# Select the columns to keep from the Expanded School Dataframe:
column_headings = ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

# Set the spending division boundaries and bin the schools by spending:
spend_divisions = [0, 600, 625, 650, 700]
spend_groups = ["<$600", "$600-625", "$625-650", ">650"]
Expanded_School_Data_DF["Budget Spent Per Student"] = pd.cut(Expanded_School_Data_DF["Per Student Budget"], spend_divisions, labels=spend_groups, include_lowest=True)
Scores_By_School_Spend_DF = Expanded_School_Data_DF.groupby("Budget Spent Per Student").mean()[column_headings]

# Format the dataframe values:
Scores_By_School_Spend_DF["Average Math Score"] = Scores_By_School_Spend_DF["Average Math Score"].map("{:.2f}".format)
Scores_By_School_Spend_DF["Average Reading Score"] = Scores_By_School_Spend_DF["Average Reading Score"].map("{:.2f}".format)
Scores_By_School_Spend_DF["% Passing Math"] = Scores_By_School_Spend_DF["% Passing Math"].map("{:.2%}".format)
Scores_By_School_Spend_DF["% Passing Reading"] = Scores_By_School_Spend_DF["% Passing Reading"].map("{:.2%}".format)
Scores_By_School_Spend_DF["% Overall Passing"] = Scores_By_School_Spend_DF["% Overall Passing"].map("{:.2%}".format)

# Display the dataframe:
styles = [dict(selector="caption", props=[("font-size","150%"), ("font-weight","bold"), ("text-align", "center"),
                                          ("background-color", "whitesmoke"), ("border-style","solid"), ("border-width", "1px")])]
title = "Average Scores and Passing Rates by School Spending"
Scores_By_School_Spend_DF.style.set_caption(title).set_table_styles(styles)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Budget Spent Per Student,,,,,
<$600,83.44,83.89,93.54%,96.46%,90.27%
$600-625,83.60,83.93,93.99%,96.54%,90.57%
$625-650,78.03,81.42,71.11%,83.45%,59.94%
>650,76.96,81.06,66.22%,81.09%,53.52%


## Average Scores and Passing Rates by School Size

In [12]:
# Select the columns to keep from the Expanded School Dataframe:
column_headings = ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

# Set the size division boundaries and bin the schools by size:
size_divisions = [0, 1000, 2000, 3000, 5000]
size_groups = ["<1000", "1000-2000", "2000-3000", ">3000"]
Expanded_School_Data_DF["Enrollment Size"] = pd.cut(Expanded_School_Data_DF["Total Students"], size_divisions, labels=size_groups, include_lowest=True)
Scores_By_School_Size_DF = Expanded_School_Data_DF.groupby("Enrollment Size").mean()[column_headings]

# Format the dataframe values:
Scores_By_School_Size_DF["Average Math Score"] = Scores_By_School_Size_DF["Average Math Score"].map("{:.2f}".format)
Scores_By_School_Size_DF["Average Reading Score"] = Scores_By_School_Size_DF["Average Reading Score"].map("{:.2f}".format)
Scores_By_School_Size_DF["% Passing Math"] = Scores_By_School_Size_DF["% Passing Math"].map("{:.2%}".format)
Scores_By_School_Size_DF["% Passing Reading"] = Scores_By_School_Size_DF["% Passing Reading"].map("{:.2%}".format)
Scores_By_School_Size_DF["% Overall Passing"] = Scores_By_School_Size_DF["% Overall Passing"].map("{:.2%}".format)

# Display the dataframe:
styles = [dict(selector="caption", props=[("font-size","150%"), ("font-weight","bold"), ("text-align", "center"),
                                          ("background-color", "whitesmoke"), ("border-style","solid"), ("border-width", "1px")])]
title = "Average Scores and Passing Rates by School Size"
Scores_By_School_Size_DF.style.set_caption(title).set_table_styles(styles)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Enrollment Size,,,,,
<1000,83.82,83.93,93.55%,96.10%,89.88%
1000-2000,83.37,83.86,93.60%,96.79%,90.62%
2000-3000,78.43,81.77,73.46%,84.47%,62.90%
>3000,77.06,80.92,66.46%,81.06%,53.67%


## Average Scores and Passing Rates by School Type

In [13]:
# Select the columns to keep from the Expanded School Dataframe:
column_headings = ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

# Group by "School Type":
Scores_By_School_Type_DF = Expanded_School_Data_DF.groupby("School Type").mean()[column_headings]

# Format the dataframe values:
Scores_By_School_Type_DF["Average Math Score"] = Scores_By_School_Type_DF["Average Math Score"].map("{:.2f}".format)
Scores_By_School_Type_DF["Average Reading Score"] = Scores_By_School_Type_DF["Average Reading Score"].map("{:.2f}".format)
Scores_By_School_Type_DF["% Passing Math"] = Scores_By_School_Type_DF["% Passing Math"].map("{:.2%}".format)
Scores_By_School_Type_DF["% Passing Reading"] = Scores_By_School_Type_DF["% Passing Reading"].map("{:.2%}".format)
Scores_By_School_Type_DF["% Overall Passing"] = Scores_By_School_Type_DF["% Overall Passing"].map("{:.2%}".format)

# Display the dataframe:
styles = [dict(selector="caption", props=[("font-size","150%"), ("font-weight","bold"), ("text-align", "center"),
                                          ("background-color", "whitesmoke"), ("border-style","solid"), ("border-width", "1px")])]
title = "Average Scores and Passing Rates by School Type"
Scores_By_School_Type_DF.style.set_caption(title).set_table_styles(styles)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%


## Final Observations

* Charter schools seem to academically outperform District schools in their Overall Passing Rates (90% versus 54%).
* Average Math and Reading Scores do not seem to vary much by grade level within each school (less than 2 points difference).
* Average Math and Reading Scores do not seem to vary much by gender within each school (less than 1 point difference); <br>however, there are slightly more schools where the girls perform better than the boys.
* Students at schools that spend less on them (<$625) seem to perform better than those at schools which spend more.
* Schools with smaller enrollments (<2000) seem to have better passing rates than the larger schools.